In [1]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [3]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [4]:
np.random.seed(42)
n =500

In [5]:
data= pd.DataFrame({
    "attendance_rate":np.random.normal(75,10,n).clip(40,100),
    "avg_marks":np.random.normal(80,40,n).clip(30,100),
    "assignment_submitted":np.random.normal(80,15,n).clip(20,100),
    "disciplinary_cases":np.random.poisson(1.2,n)
})

In [10]:
data

,attendance_rate,avg_marks,assignment_submitted,disciplinary_cases
0,79.967142,117.047102,100.000000,0
1,73.617357,156.376666,93.869505,2
2,81.476885,24.057297,80.894456,0
3,90.230299,102.518769,70.295948,1
4,72.658466,53.974297,90.473350,2
...,...,...,...,...
495,80.389100,68.755988,100.000000,2
496,64.627538,151.907461,100.000000,0
497,73.096613,105.633714,98.125493,0
498,66.243817,57.152840,95.360938,2


In [6]:
data["dropout"] =(
    (data["attendance_rate"]<60) |
    (data["avg_marks"]<50) |
    (data["disciplinary_cases"]>3)
).astype(int)

In [19]:
X = data.drop("dropout", axis=1)

In [14]:
y= data["dropout"]

In [21]:
X

,attendance_rate,avg_marks,assignment_submitted,disciplinary_cases
0,79.967142,100.000000,100.000000,0
1,73.617357,100.000000,93.869505,2
2,81.476885,30.000000,80.894456,0
3,90.230299,100.000000,70.295948,1
4,72.658466,53.974297,90.473350,2
...,...,...,...,...
495,80.389100,68.755988,100.000000,2
496,64.627538,100.000000,100.000000,0
497,73.096613,100.000000,98.125493,0
498,66.243817,57.152840,95.360938,2


In [16]:
y

0      0
1      0
2      1
3      0
4      0
      ..
495    0
496    0
497    0
498    0
499    0
Name: dropout, Length: 500, dtype: int32

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
X_train

,attendance_rate,avg_marks,assignment_submitted,disciplinary_cases
227,64.222552,93.903268,77.794970,4
417,76.156746,87.464365,84.028884,0
203,85.538021,30.000000,88.149470,1
126,65.094637,40.415807,84.539532,2
329,81.283455,68.728616,88.998931,1
...,...,...,...,...
106,93.861859,49.048432,89.191609,1
270,89.412733,100.000000,96.369652,1
348,67.815558,54.490401,59.496820,2
435,75.740948,59.917831,84.238699,0


In [24]:
X_test

,attendance_rate,avg_marks,assignment_submitted,disciplinary_cases
361,90.327389,30.000000,88.689372,1
73,90.646437,71.235979,60.755117,2
374,96.531825,78.674921,100.000000,1
155,67.856486,82.368737,95.147256,0
104,73.387143,96.537396,96.622742,0
...,...,...,...,...
220,98.146586,92.312071,62.186024,0
176,75.130019,35.576966,54.605648,2
320,75.969960,59.308462,80.849749,0
153,77.322537,35.738964,60.153653,1


In [26]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [27]:
models ={
    "LogisticRegression":LogisticRegression(max_iter=1000),
    "Decision Tree":DecisionTreeClassifier(max_depth=5),
    "Random Forest":RandomForestClassifier(n_estimators=100),
    "SVM":SVC(probability=True)
}

In [31]:
result=[]
for name, model in models.items():
    if name in ["LogisticRegression","SVM"]:
        model.fit(X_train_scaled, y_train)
        y_pred= model.predict(X_test_scaled)
        y_prob= model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train,y_train)
        y_pred= model.predict(X_test)
        y_prob= model.predict_proba(X_test)[:, 1]


In [32]:
cm= confusion_matrix(y_test,y_pred)

In [36]:
result.append({
    "Model":name,
    "Accuracy":accuracy_score(y_test,y_pred),
    "Precision":precision_score(y_test,y_pred),
    "Recall":recall_score(y_test,y_pred),
    "f1_score":f1_score(y_test,y_pred),
    "roc_auc_score":roc_auc_score(y_test,y_pred),
    "confusion_matrix":cm
})

In [37]:
result_df=pd.DataFrame(result)
print(result_df)

  Model  Accuracy  Precision    Recall  f1_score  roc_auc_score  \
0   SVM     0.944        1.0  0.810811  0.895522       0.905405   

     confusion_matrix  
0  [[88, 0], [7, 30]]  


In [39]:
for r in result:
    print(f"\n{r['Model']} Confusion Matrix:")
    print(r["confusion_matrix"])


SVM Confusion Matrix:
[[88  0]
 [ 7 30]]
